# <center> STA663 Final Project: Scalable K-means
### <center> Xin Xu, Fu Wen

## <center> Abstract 

The k-means algorithm is one of the most famous machine learning algorithms used to cluster data. The k-means++ algorithm is developed from it and find a better set of initialization centers to avoid the problem of locally blocking when finding global optimal solution. However, K-means++ has a limited applicability to big data due to its sequencely initialization process. The K-means|| algorithm in the paper ”Scalable K-Means++” [1] further develops the k-means++ algorithm for scalable data. In this report, we implemented the K-means|| algorithm, developed Python code, parallelized the algorithm in Spark, applied the algorithm to GAUSSMIXTURE dataset, and compared the solution and convergence speed with k-means|| algorithm to the result of k-means and k-means++ algorithm. 


## 1. Introduction

### 1.1 Background and Related Algorithm

As one of the most popular clustering algorithm, k-means algorithm is used widely for the last half of the century. The main idea is to randomly choose k centers, repeatedly assign each point to its nearest center and calculate the new centers by minimizing the sum of squares of the distance in its neighborhood.

K-means algorithm has a critical problem  of improper initialization. The algorithm with an improper initialization cannot find a global optimal solution but a locally optimal and the running time before convergence is long. K-means++ algorithm avoids this problem by finding k better initial centers. It first samples one random point uniformly from the data, then sets the subsequent k-1 centers with probability proportional to its contribution to the overall error given the previous centers. It reduces the probability of picking several initial centers in one cluster, which improves the final solution of k-means algorithm. However, the sequent initialization process also limits its applicability to big data or data with large k since it is not scalable.


Bahmani, Moseley, Vattani, Kumar and Vassilvitskii constructed the scalable k-means++ algorithm (k-means|| algorithm) in their paper "Scalable K-Means++"[1]. The main idea is to sample more than one point ($O(k)$) in each round and repeat the process for less iterations ($O(log n)$). Then, it reclusters the $O(k log n)$ points generated from above process into k initial centers. 


### 1.2 Notation and the Algorithm

Suppose $X = \{x_1, \dots, x_n \}$ are d-dimentional points to be clustered and k is the number of cluster (a positer integer). 

For a subset $Y \in X$, define the distance from a point $x$ to $Y$ as $d(x,Y) = min_{y \in Y} \| x-y\|$, where \|x -y \| denote the Euclidean distance between $x$ and $y$, define the centroid of $Y$ as
$$Centroid(Y) = \frac{1}{|Y|} \sum_{y \in Y} y$$

For a set of cluster centers $C = \{ c_1, c_2 ,\dots , c_k\}$, define the _cost of $X$_ with respect to $C$ as:

$$ \varphi_X(C) = \sum_{x \in X} d(x,C)^2$$

In k-means|| algorithm, it set an oversampling factor $l = \Omega (k)$. $l>1$ is an integer.



## Implementation

## Testing

## Optimization

## High performance computing

## Application and comparison